## 数据处理, 数据保存到npz中

1. train, vali, testa 的词id np array
2. train, vali, test 的标签保存为 np array
3. word2id保存为json
4. embedding 保存为 np array

In [ ]:
import pandas as pd
import numpy as np
EMBEDDING_FILE = '../inputs/fasttextwordvec.vec'
train = pd.read_csv("../inputs/train.csv")
test = pd.read_csv("../inputs/testa.csv")
val = pd.read_csv("../inputs/vali.csv")
X_train = train["content"].fillna("无").str.lower()
X_val = val["content"].fillna("无").str.lower()
X_test = test["content"].fillna("无").str.lower()

In [ ]:
import pickle

## 处理词向量

In [ ]:
word2id = pickle.load(open("../inputs/word2id.pkl", 'rb'))

In [ ]:
# 数据参数
max_features=65462
maxlen=200
embed_size=300

In [ ]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
word_index = word2id
#prepare embedding matrix
num_words = max_features
    

In [ ]:
embedding_matrix = np.random.randn(num_words, embed_size) * 0.01
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector  

In [ ]:
# 保存embedding
np.savez_compressed("../inputs/fasttextwordvec.npz",vector=embedding_matrix)

## 保存Y

In [ ]:
from tensorflow.keras.utils import to_categorical

from typing import List

columns = train.columns.tolist()[2:]
def get_y(df:pd.DataFrame, cols:List[str]=columns) -> List[np.array]:
    y_dict = dict()
    for col in cols:
        y = df[col].values + 2
        y_ = to_categorical(y, num_classes=4)
        y_dict[col] = y_
    return y_dict

In [ ]:
y_train = get_y(train)
y_val = get_y(val)

In [ ]:
np.savez_compressed("../inputs/Y_train.npz",**y_train)
np.savez_compressed("../inputs/Y_valid.npz",**y_val)

In [ ]:
pickle.dump(columns, open("../inputs/columns.pkl", 'wb'))

## 保存X

In [ ]:
import tensorflow as tf

def transform_data_to_id(x_arr, word2id):
    data = []

    def map_word_to_id(word):
        output = []
        if word in word2id:
            output.append(word2id[word])
        else:
            chars = list(word)
            for char in chars:
                if char in word2id:
                    output.append(word2id[char])
                else:
                    output.append(1)
        return output

    def map_sent_to_id(sent):
        output = []
        for word in sent:
            output.extend(map_word_to_id(word))
        return output
    for s in x_arr:
        data.append(map_sent_to_id(s))
    
    return data

In [ ]:
x_val = transform_data_to_id(X_val, word2id)

In [ ]:
import keras
pad_sequence = keras.preprocessing.sequence.pad_sequences

In [ ]:
X_val = transform_data_to_id(X_val, word2id)
X_val = pad_sequence(X_val,maxlen=maxlen,padding='pre',truncating='pre',value = 0)
X_train = transform_data_to_id(X_train, word2id)
X_train = pad_sequence(X_train,maxlen=maxlen,padding='pre',truncating='pre',value = 0)
X_test = transform_data_to_id(X_test, word2id)
X_test = pad_sequence(X_test,maxlen=maxlen,padding='pre',truncating='pre',value = 0)

In [ ]:
np.savez_compressed("../inputs/X_train.npz",X = X_train)
np.savez_compressed("../inputs/X_valid.npz",X = X_val)
np.savez_compressed("../inputs/X_test.npz",X = X_test)

In [ ]:
columns

## 模型

In [1]:

import torch as t
import time

class BasicModule(t.nn.Module):
    '''
    封装了nn.Module,主要是提供了save和load两个方法
    '''

    def __init__(self):
        super(BasicModule,self).__init__()
        self.model_name=str(type(self))# 默认名字

    def load(self, path,change_opt=True):
        print("Loading model from " + path)
        data = t.load(path)
        self.load_state_dict(data)
        return self.cuda()

    def save(self, name=None,new=False):
        prefix = '../ckpt/' + self.model_name + '_'
        if name is None:
            name = time.strftime('%m%d_%H:%M:%S.pth')
        path = prefix+name +'.pt'
        data=self.state_dict()
        t.save(data, path)
        print("Saving model to "+ path)
        return path

    def get_optimizer(self,lr1,lr2=0,weight_decay = 0):
        ignored_params = list(map(id, self.encoder.parameters()))
        base_params = filter(lambda p: id(p) not in ignored_params,
                        self.parameters())
        if lr2 is None: lr2 = lr1*0.5 
        optimizer = t.optim.Adam([
                dict(params=base_params,weight_decay = weight_decay,lr=lr1),
                {'params': self.encoder.parameters(), 'lr': lr2}
            ])
        return optimizer


In [2]:
from torch import nn
from collections import OrderedDict

class Inception(nn.Module):
    def __init__(self,cin,co,relu=True,norm=True):
        super(Inception, self).__init__()
        assert(co%4==0)
        cos=[co//4]*4
        self.activa=nn.Sequential()
        if norm:self.activa.add_module('norm',nn.BatchNorm1d(co))
        if relu:self.activa.add_module('relu',nn.ReLU(True))
        self.branch1 =nn.Sequential(OrderedDict([
            ('conv1', nn.Conv1d(cin,cos[0], 1,stride=1)),
            ])) 
        self.branch2 =nn.Sequential(OrderedDict([
            ('conv1', nn.Conv1d(cin,cos[1], 1)),
            ('norm1', nn.BatchNorm1d(cos[1])),
            ('relu1', nn.ReLU(inplace=True)),
            ('conv3', nn.Conv1d(cos[1],cos[1], 3,stride=1,padding=1)),
            ]))
        self.branch3 =nn.Sequential(OrderedDict([
            ('conv1', nn.Conv1d(cin,cos[2], 3,padding=1)),
            ('norm1', nn.BatchNorm1d(cos[2])),
            ('relu1', nn.ReLU(inplace=True)),
            ('conv3', nn.Conv1d(cos[2],cos[2], 5,stride=1,padding=2)),
            ]))
        self.branch4 =nn.Sequential(OrderedDict([
            #('pool',nn.MaxPool1d(2)),
            ('conv3', nn.Conv1d(cin,cos[3], 3,stride=1,padding=1)),
            ]))
    def forward(self,x):
        branch1=self.branch1(x)
        branch2=self.branch2(x)
        branch3=self.branch3(x)
        branch4=self.branch4(x)
        result=self.activa(torch.cat((branch1,branch2,branch3,branch4),1))
        return result
    
    


In [3]:
import copy
import pickle as pkl
class CNNText_inception(BasicModule):
    def __init__(self, opt):
        super(CNNText_inception, self).__init__()
        incept_dim=opt.inception_dim
        self.label_list = pkl.load(open(opt.label_list_pkl, "rb"))
        self.model_name = 'CNNText_inception'
        self.opt=opt
        self.encoder = nn.Embedding(opt.vocab_size,opt.embedding_dim)
        self.content_conv=nn.Sequential(
            Inception(opt.embedding_dim,incept_dim),#(batch_size,64,opt.content_seq_len)->(batch_size,64,(opt.content_seq_len)/2)
            #Inception(incept_dim,incept_dim),#(batch_size,64,opt.content_seq_len/2)->(batch_size,32,(opt.content_seq_len)/4)
            Inception(incept_dim,incept_dim),
            nn.MaxPool1d(opt.content_seq_len)
        )
        self.fc_sub = nn.Sequential(
            nn.Linear(incept_dim,opt.linear_hidden_size),
            nn.BatchNorm1d(opt.linear_hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(opt.linear_hidden_size,opt.num_classes)
        )
        self.fc = nn.ModuleDict({label: copy.deepcopy(self.fc_sub) for label in self.label_list}
        )
        if opt.embedding_path:
            print('load embedding')
            self.encoder.weight.data.copy_(t.from_numpy(np.load(opt.embedding_path)['vector']))
    def forward(self, content):
        content = self.encoder(content)
        content_out=self.content_conv(content.permute(0,2,1))        
        out = content_out.view(content_out.size(0), -1)
        out_dict = {label: self.fc[label](out) for label in self.label_list}
        return out_dict

In [4]:
import torch as t
import numpy as np
from torch import nn


def kmax_pooling(x, dim, k):
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

class RCNN(BasicModule): 
    def __init__(self, opt ):
        super(RCNN, self).__init__()
        self.model_name = 'RCNN'
        self.opt=opt
        self.label_list = pkl.load(open(opt.label_list_pkl, "rb"))
        kernel_size = opt.kernel_size
        self.encoder = nn.Embedding(opt.vocab_size,opt.embedding_dim)

        self.title_lstm = nn.LSTM(input_size = opt.embedding_dim,\
                            hidden_size = opt.hidden_size,
                            num_layers = opt.num_layers,
                            bias = True,
                            batch_first = False,
                            dropout = 0.5,
                            bidirectional = True
                            )

        self.content_lstm =nn.LSTM(input_size = opt.embedding_dim,\
                            hidden_size = opt.hidden_size,
                            num_layers = opt.num_layers,
                            bias = True,
                            batch_first = False,
                            # dropout = 0.5,
                            bidirectional = True
                            )

        self.content_conv = nn.Sequential(
            nn.Conv1d(in_channels = opt.hidden_size*2 + opt.embedding_dim,
                      out_channels = opt.content_dim,
                      kernel_size =  kernel_size),
            nn.BatchNorm1d(opt.content_dim),
            nn.ReLU(inplace=True),

            nn.Conv1d(in_channels = opt.content_dim,
                      out_channels = opt.content_dim,
                      kernel_size =  kernel_size),
            nn.BatchNorm1d(opt.content_dim),
            nn.ReLU(inplace=True),
        )

        self.fc_sub = nn.Sequential(
            nn.Linear(opt.kmax_pooling*(opt.content_dim),opt.linear_hidden_size),
            nn.BatchNorm1d(opt.linear_hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(opt.linear_hidden_size,opt.num_classes)
        )
        self.fc = nn.ModuleDict({label: copy.deepcopy(self.fc_sub) for label in self.label_list}
        )
        if opt.embedding_path:
            self.encoder.weight.data.copy_(t.from_numpy(np.load(opt.embedding_path)['vector']))
            
    def forward(self, content):
        content = self.encoder(content)

        content_out = self.content_lstm(content.permute(1,0,2))[0].permute(1,2,0)
        content_em = (content).permute(0,2,1)
        content_out = t.cat((content_out,content_em),dim=1)
        content_conv_out = kmax_pooling(self.content_conv(content_out),2,self.opt.kmax_pooling)
        out = content_conv_out.view(content_conv_out.size(0), -1)
        out_dict = {label: self.fc[label](out) for label in self.label_list}
        return out_dict

## config

In [5]:
#coding:utf8
import time
import warnings

class ModelConfig(object):
    '''
    并不是所有的配置都生效,实际运行中只根据需求获取自己需要的参数
    '''
    content_dim = 200 #描述的卷积核数
    num_classes = 4 # 类别
    embedding_dim = 300 # embedding大小
    linear_hidden_size = 1024 # 全连接层隐藏元数目
    kmax_pooling = 2# k
    hidden_size = 256 #LSTM hidden size
    num_layers=2 #LSTM layers
    inception_dim = 512 #inception的卷积核数
    
    vocab_size = 65462
    kernel_size = 3 #单尺度卷积核
    kernel_sizes = [2,3,4] #多尺度卷积核
    content_seq_len = 200 
    label_list_pkl = "../inputs/columns.pkl"
    embedding_path = '../inputs/fasttextwordvec.npz' # Embedding
    
    
class ModelConfig2(object):
    '''
    并不是所有的配置都生效,实际运行中只根据需求获取自己需要的参数
    '''
    content_dim = 100 #描述的卷积核数
    num_classes = 4 # 类别
    embedding_dim = 300 # embedding大小
    linear_hidden_size = 128 # 全连接层隐藏元数目
    kmax_pooling = 2# k
    hidden_size = 128 #LSTM hidden size
    num_layers=2 #LSTM layers
    inception_dim = 128 #inception的卷积核数

    vocab_size = 65462
    kernel_size = 3 #单尺度卷积核
    kernel_sizes = [2,3,4] #多尺度卷积核
    content_seq_len = 200 
    label_list_pkl = "../inputs/columns.pkl"
    embedding_path = '../inputs/fasttextwordvec.npz' # Embedding

In [6]:
# modelopt = ModelConfig()
modelopt = ModelConfig()

In [7]:
import numpy as np
import pickle as pkl
model = RCNN(modelopt)

## Dataset

In [8]:
from torch.utils import data
## 数据集，对于训练集生成X:wordid_list,Y:dict of label one hot vector
## 对于验证集只生成 X, Y直接通过Dataset.Y[label]获取相应的标签
## 训练数据同时有数据集增强：随机dropout,打散顺序

class FGSentimetDataset(data.Dataset):
    def __init__(self, X_npz, Y_npz, label_pkl, augument=False, training=False, dropout_rate=0.3, augument_rate=0.4):
        self.dropout_rate = dropout_rate
        self.augument_rate = augument_rate
        self.augument=augument
        self.training = training
        self.label_list = pickle.load(open(label_pkl, 'rb'))
        self.X = np.load(X_npz)['X']
        dataset = np.load(Y_npz)
        self.Y = {}
        for col in self.label_list:
            self.Y[col] = dataset[col]
        self._len = self.X.shape[0]
    def shuffle(self,d):
        return np.random.permutation(d.tolist())

    def dropout(self,d,p=0.5):
        len_ = len(d)
        index = np.random.choice(len_,int(len_*p))
        d[index]=0
        return d     


    def __getitem__(self,index):
        
        content =  self.X[index]
    
        if self.training:  
            if self.augument :
                augument=random.random()

                if augument>self.augument_rate:
                    content = self.dropout(content,p=self.dropout_rate)
                else:
                    content = self.shuffle(content)

            data =t.from_numpy(content).long()
            label_dict = {label:t.from_numpy(self.Y[label][index]).long() for label in self.label_list}
            return data, label_dict
        else:
            return t.from_numpy(content).long()

    def __len__(self):
        return self._len    
  

## 验证函数

In [9]:
from collections import defaultdict  
from sklearn import metrics
import torch
import torch.utils.data
import json

In [10]:
import pickle
loss_function = nn.CrossEntropyLoss()
Train_dataset = FGSentimetDataset("../inputs/X_train.npz","../inputs/Y_train.npz", "../inputs/columns.pkl", augument=False, training=True)

In [11]:
Valid_dataset =  FGSentimetDataset("../inputs/X_valid.npz","../inputs/Y_valid.npz", "../inputs/columns.pkl", augument=False, training=False)


In [12]:
trainloader = torch.utils.data.DataLoader(Train_dataset,
                    batch_size = 512,
                    shuffle = True,
                    num_workers = 8,
                    pin_memory = True
                    )

In [13]:
validloader = torch.utils.data.DataLoader(Valid_dataset,
                    batch_size = 512,
                    shuffle = False,
                    num_workers = 1,
                    pin_memory = True
                    )


In [14]:
def eval_metrics(y_pred_dict, y_true_dict):
    accuracys = {}
    f1s = {}
    for col, y_pred in y_pred_dict.items():
        # accuracys[col] = metrics.accuracy_score(y_true_dict[col], y_pred)
        f1s[col] = metrics.f1_score(y_true_dict[col], y_pred, average='macro')

    return f1s

def eval(model, valiloader,y_true, use_cuda=True):
    
    model.eval()
    y_true_ = {}
    y_pred = defaultdict(list)
    for bx in validloader:
        if use_cuda:
            bx =  bx.cuda()
        y_pre = model(bx)
        for col in model.label_list:
            y_label = torch.max(y_pre[col], 1)[1].data
            y_pred[col].extend(y_label.tolist())
            y_true_[col] = np.argmax(y_true[col], 1)
    f1s = eval_metrics(y_pred, y_true_)
    print(json.dumps(f1s, indent=4, sort_keys=True))
    return np.mean(list(f1s.values()))

#scores = eval(model,validloader, Valid_dataset.Y)

In [16]:
import tqdm
import random
best_score = 0
lr = 5e-3
lr2 = 1e-3
lr_decay = 0.9
early_stops = 20
i = 0
optimizer = model.get_optimizer(lr, lr2,lr_decay)
model.cuda()

for epoch in range(1000):
    model.train()
    print(f"training epoch {epoch}")
    for ii,(content, true_dict) in enumerate(trainloader):
        content = content.cuda()
        optimizer.zero_grad()
        pred_dict = model(content)
        loss = torch.mean(torch.stack([loss_function(pred, torch.max(true_dict[col].cuda(), 1)[1]) for col, pred in pred_dict.items()]))
        loss.backward()
        optimizer.step()
    
    scores = eval(model,validloader, Valid_dataset.Y)
    print(f"epoch: {epoch} LR: {lr}, F1_score:{scores}")
    if scores>best_score:
        print(f"F1-score improved from {best_score} to {scores}")
        i = 0
        best_score = scores
        best_path = model.save(name = str(scores),new=True)

    if scores < best_score:
        i += 1
        print(f"F1-score did not improved from {best_score} for {i} epochs")       
        model.load(best_path,change_opt=False)
        if i > early_stops:
            print("Stop training")
            break
        lr = lr * lr_decay
        lr2= 2e-4 if lr2==0 else  lr2*0.8
        optimizer = model.get_optimizer(lr,lr2,0)                        


training epoch 0
{
    "dish_look": 0.2088283251805264,
    "dish_portion": 0.17549325025960538,
    "dish_recommendation": 0.22307351475095077,
    "dish_taste": 0.17168621957625635,
    "environment_cleaness": 0.19507236949097415,
    "environment_decoration": 0.17106705846234815,
    "environment_noise": 0.20612436816739158,
    "environment_space": 0.19376505655138623,
    "location_distance_from_business_district": 0.22255105060668837,
    "location_easy_to_find": 0.21715190828179212,
    "location_traffic_convenience": 0.21969951788317077,
    "others_overall_experience": 0.20089730807577266,
    "others_willing_to_consume_again": 0.19204237496920423,
    "price_cost_effective": 0.21621008021795066,
    "price_discount": 0.19087461874536316,
    "price_level": 0.16662221629550608,
    "service_parking_convenience": 0.241788886593679,
    "service_serving_speed": 0.22901687321602773,
    "service_wait_time": 0.2343910472075645,
    "service_waiters_attitude": 0.14232915255853879
}

/home/xq/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


{
    "dish_look": 0.2088283251805264,
    "dish_portion": 0.17549325025960538,
    "dish_recommendation": 0.22307351475095077,
    "dish_taste": 0.17168621957625635,
    "environment_cleaness": 0.19507236949097415,
    "environment_decoration": 0.17106705846234815,
    "environment_noise": 0.20612436816739158,
    "environment_space": 0.19376505655138623,
    "location_distance_from_business_district": 0.22255105060668837,
    "location_easy_to_find": 0.21715190828179212,
    "location_traffic_convenience": 0.21969951788317077,
    "others_overall_experience": 0.20089730807577266,
    "others_willing_to_consume_again": 0.19204237496920423,
    "price_cost_effective": 0.21621008021795066,
    "price_discount": 0.19087461874536316,
    "price_level": 0.16662221629550608,
    "service_parking_convenience": 0.241788886593679,
    "service_serving_speed": 0.22901687321602773,
    "service_wait_time": 0.2343910472075645,
    "service_waiters_attitude": 0.14272103658536586
}
epoch: 1 LR: 0.0

{
    "dish_look": 0.316883549006125,
    "dish_portion": 0.4941899666474814,
    "dish_recommendation": 0.5144854020988627,
    "dish_taste": 0.5562640512320665,
    "environment_cleaness": 0.34955538304259787,
    "environment_decoration": 0.34251539616333915,
    "environment_noise": 0.4370107570852388,
    "environment_space": 0.33179058705664016,
    "location_distance_from_business_district": 0.2460322222700455,
    "location_easy_to_find": 0.22314187486292547,
    "location_traffic_convenience": 0.29342910264784616,
    "others_overall_experience": 0.5362668187583955,
    "others_willing_to_consume_again": 0.6249075971768121,
    "price_cost_effective": 0.5503905846351376,
    "price_discount": 0.39275777827612524,
    "price_level": 0.5769859114166412,
    "service_parking_convenience": 0.3149818154457291,
    "service_serving_speed": 0.4359886778547755,
    "service_wait_time": 0.2343910472075645,
    "service_waiters_attitude": 0.45168512618924717
}
epoch: 8 LR: 0.00364500000

{
    "dish_look": 0.34659368446419303,
    "dish_portion": 0.42903069016529627,
    "dish_recommendation": 0.4686564215218038,
    "dish_taste": 0.4347770148445932,
    "environment_cleaness": 0.4304929392617661,
    "environment_decoration": 0.36132639999748617,
    "environment_noise": 0.47598153601263143,
    "environment_space": 0.33882912646026936,
    "location_distance_from_business_district": 0.22736998231688732,
    "location_easy_to_find": 0.2423066751320959,
    "location_traffic_convenience": 0.26864874505027664,
    "others_overall_experience": 0.4379972169971509,
    "others_willing_to_consume_again": 0.61970604970886,
    "price_cost_effective": 0.5758560247089171,
    "price_discount": 0.4001540001305378,
    "price_level": 0.5966445430238335,
    "service_parking_convenience": 0.3177328640246995,
    "service_serving_speed": 0.4078040795524312,
    "service_wait_time": 0.2859130314559091,
    "service_waiters_attitude": 0.4973331999901638
}
epoch: 15 LR: 0.00239148450

...

{
    "dish_look": 0.36618766109311846,
    "dish_portion": 0.5145561200701898,
    "dish_recommendation": 0.5579399971960476,
    "dish_taste": 0.5415849276934579,
    "environment_cleaness": 0.47518141621515964,
    "environment_decoration": 0.387274452044407,
    "environment_noise": 0.4909763220459553,
    "environment_space": 0.3802027583158073,
    "location_distance_from_business_district": 0.22361712418366267,
    "location_easy_to_find": 0.23825304645371198,
    "location_traffic_convenience": 0.3082826230362736,
    "others_overall_experience": 0.5318780560122187,
    "others_willing_to_consume_again": 0.6715283076922727,
    "price_cost_effective": 0.5738703897579833,
    "price_discount": 0.4358843591454745,
    "price_level": 0.6185763239587089,
    "service_parking_convenience": 0.3518183348124731,
    "service_serving_speed": 0.4963127719324303,
    "service_wait_time": 0.3101844089199512,
    "service_waiters_attitude": 0.5931200589100523
}
epoch: 36 LR: 0.0004923854510

{
    "dish_look": 0.36669235840392994,
    "dish_portion": 0.5193294780800262,
    "dish_recommendation": 0.5594608742205134,
    "dish_taste": 0.5492332360653599,
    "environment_cleaness": 0.4840838837136653,
    "environment_decoration": 0.4010061801010889,
    "environment_noise": 0.4818438903078371,
    "environment_space": 0.39524664656594133,
    "location_distance_from_business_district": 0.23257067832469386,
    "location_easy_to_find": 0.2464902069790165,
    "location_traffic_convenience": 0.317603277261078,
    "others_overall_experience": 0.5299921236027065,
    "others_willing_to_consume_again": 0.6605008014183851,
    "price_cost_effective": 0.6144551666989706,
    "price_discount": 0.42840173963464356,
    "price_level": 0.617555191467829,
    "service_parking_convenience": 0.35618343829881455,
    "service_serving_speed": 0.5038090952941834,
    "service_wait_time": 0.3149903136328143,
    "service_waiters_attitude": 0.595514920384979
}
epoch: 43 LR: 0.00029074868501

{
    "dish_look": 0.35702750192605925,
    "dish_portion": 0.511500846921676,
    "dish_recommendation": 0.5614076387060281,
    "dish_taste": 0.5616894917167359,
    "environment_cleaness": 0.4839110397680866,
    "environment_decoration": 0.39689824078236385,
    "environment_noise": 0.4857110465641749,
    "environment_space": 0.3922342241431712,
    "location_distance_from_business_district": 0.23071006379578662,
    "location_easy_to_find": 0.2429205846131014,
    "location_traffic_convenience": 0.3091121477432277,
    "others_overall_experience": 0.5314721423254771,
    "others_willing_to_consume_again": 0.6771619926219465,
    "price_cost_effective": 0.6032251101941124,
    "price_discount": 0.42835901195941384,
    "price_level": 0.6166525605343239,
    "service_parking_convenience": 0.3519593602662713,
    "service_serving_speed": 0.49944815046139,
    "service_wait_time": 0.31345985786533914,
    "service_waiters_attitude": 0.5940408908914749
}
epoch: 50 LR: 0.00017168419101

{
    "dish_look": 0.3670756811874785,
    "dish_portion": 0.510733564805329,
    "dish_recommendation": 0.5503274742476665,
    "dish_taste": 0.5476605372452522,
    "environment_cleaness": 0.4851838455032494,
    "environment_decoration": 0.39990004055240447,
    "environment_noise": 0.4850583342685929,
    "environment_space": 0.3945153159267014,
    "location_distance_from_business_district": 0.2314950968710195,
    "location_easy_to_find": 0.2454853773980822,
    "location_traffic_convenience": 0.3242518599722576,
    "others_overall_experience": 0.5311488097835031,
    "others_willing_to_consume_again": 0.6706521777766274,
    "price_cost_effective": 0.6107165557665161,
    "price_discount": 0.43174643161673076,
    "price_level": 0.6167939192530889,
    "service_parking_convenience": 0.36223652053808536,
    "service_serving_speed": 0.49245136454071714,
    "service_wait_time": 0.29828767441168097,
    "service_waiters_attitude": 0.6032815354824019
}
epoch: 57 LR: 8.211601634130

{
    "dish_look": 0.36954351952116476,
    "dish_portion": 0.5121441846633956,
    "dish_recommendation": 0.5541811148045808,
    "dish_taste": 0.5479192515577106,
    "environment_cleaness": 0.48176772050465566,
    "environment_decoration": 0.4000762580059185,
    "environment_noise": 0.4847610444523178,
    "environment_space": 0.38933729646898463,
    "location_distance_from_business_district": 0.23270532841430677,
    "location_easy_to_find": 0.24587367985498157,
    "location_traffic_convenience": 0.3238807532236855,
    "others_overall_experience": 0.5325935442905111,
    "others_willing_to_consume_again": 0.6706503023699233,
    "price_cost_effective": 0.6079059557550373,
    "price_discount": 0.4297468660035011,
    "price_level": 0.6200391497781961,
    "service_parking_convenience": 0.35734763076996184,
    "service_serving_speed": 0.4923588238470213,
    "service_wait_time": 0.3069286238373213,
    "service_waiters_attitude": 0.600715134909523
}
epoch: 64 LR: 3.92758360563

In [17]:
test_arr = np.load("../inputs/X_test.npz")

In [18]:
test_arr['X']

array([[6631, 1369,    1, ...,  487,    1,    2],
       [   8,    1,  644, ...,  147,    1,    2],
       [   1,  430,    1, ...,  479,    1,    2],
       ...,
       [   1,  152,   22, ...,  644,    1,    2],
       [   1,  379,  152, ...,  147,    1,    2],
       [   1, 1025, 8134, ...,  147,    1,    2]], dtype=int32)

In [19]:
test_loader = data.DataLoader(torch.from_numpy(test_arr['X']).long(), batch_size=1024, shuffle=False, num_workers=8)

In [20]:
def pred_test(model, testloader, use_cuda=True):
    model.eval()
    y_pred = defaultdict(list)
    for bx in testloader:
        if use_cuda:
            bx = bx.cuda()
        y_pre = model(bx)
        for col in model.label_list:
            y_label = torch.max(y_pre[col], 1)[1].data
            y_pred[col].extend(y_label.tolist())
    return y_pred
            



In [21]:
pred_test = pred_test(model, test_loader)

In [22]:
import pandas as pd
def load_data_from_csv(file_name, header=0, encoding="utf-8"):

    data_df = pd.read_csv(file_name, header=header, encoding=encoding)

    return data_df

In [23]:
test = load_data_from_csv("../inputs/sentiment_analysis_testa.csv")

In [24]:
for col, pred in pred_test.items():
    test[col] = pred
    test[col] -= 2

In [25]:
test.to_csv("../output/RCNN1.csv", encoding="utf_8_sig", index=False)